In [1]:
from distributed import Client

import datatree
import intake

In [2]:
client = Client(processes=False)
client

/home/andersy005/mambaforge/envs/playground/lib/python3.10/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41017 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.0.0.253:41017/status,
Dashboard: http://10.0.0.253:41017/status,Workers: 1
Total threads: 24,Total memory: 31.27 GiB
Status: running,Using processes: False
Comm: inproc://10.0.0.253/2053943/1,Workers: 1
Dashboard: http://10.0.0.253:41017/status,Total threads: 24
Started: Just now,Total memory: 31.27 GiB
Comm: inproc://10.0.0.253/2053943/4,Total threads: 24
Dashboard: http://10.0.0.253:40729/status,Memory: 31.27 GiB
Nanny: None,


In [3]:
cat = intake.open_esm_datastore(
    'https://raw.githubusercontent.com/NCAR/cesm2-le-aws/main/intake-catalogs/aws-cesm2-le.json',
    sep='/',
)
cat

,unique
variable,53
long_name,51
component,4
experiment,2
forcing_variant,2
frequency,3
vertical_levels,3
spatial_domain,3
units,20
start_time,4


In [4]:
cat.keys()

dict_keys(['atm/historical/daily/cmip6', 'atm/historical/daily/smbb', 'atm/historical/monthly/cmip6', 'atm/historical/monthly/smbb', 'atm/historical/static/cmip6', 'atm/historical/static/smbb', 'atm/ssp370/daily/cmip6', 'atm/ssp370/daily/smbb', 'atm/ssp370/monthly/cmip6', 'atm/ssp370/monthly/smbb', 'atm/ssp370/static/cmip6', 'atm/ssp370/static/smbb', 'ice/historical/daily/cmip6', 'ice/historical/daily/smbb', 'ice/historical/monthly/cmip6', 'ice/historical/monthly/smbb', 'ice/historical/static/cmip6', 'ice/historical/static/smbb', 'ice/ssp370/daily/cmip6', 'ice/ssp370/daily/smbb', 'ice/ssp370/monthly/cmip6', 'ice/ssp370/monthly/smbb', 'ice/ssp370/static/cmip6', 'ice/ssp370/static/smbb', 'lnd/historical/daily/cmip6', 'lnd/historical/daily/smbb', 'lnd/historical/monthly/cmip6', 'lnd/historical/monthly/smbb', 'lnd/ssp370/daily/cmip6', 'lnd/ssp370/daily/smbb', 'lnd/ssp370/monthly/cmip6', 'lnd/ssp370/monthly/smbb', 'ocn/historical/monthly/cmip6', 'ocn/historical/monthly/smbb', 'ocn/historica

In [5]:
cat_sub = cat.search(frequency='monthly', variable=['TREFHT', 'O2', 'PRECC'])
dsets = cat_sub.to_dataset_dict(storage_options={'anon': True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component/experiment/frequency/forcing_variant'


In [6]:
results = {}

for key, ds in dsets.items():
    subset = {}
    if 'z_t' in ds:
        subset['z_t'] = [0]
    subset['time'] = list(range(6))
    subset['member_id'] = [0]

    dset_subset = ds.isel(**subset).load()
    del dset_subset.attrs['intake_esm_varname']
    results[key] = dset_subset

results['ocn/ssp370/monthly/cmip6']

<xarray.Dataset>
Dimensions:     (member_id: 1, time: 6, z_t: 1, nlat: 384, nlon: 320, d2: 2)
Coordinates:
  * member_id   (member_id) <U12 'r10i1181p1f1'
  * time        (time) object 2015-01-16 12:00:00 ... 2015-06-16 00:00:00
    time_bound  (time, d2) object 2015-01-01 00:00:00 ... 2015-07-01 00:00:00
  * z_t         (z_t) float32 500.0
Dimensions without coordinates: nlat, nlon, d2
Data variables:
    O2          (member_id, time, z_t, nlat, nlon) float32 nan nan ... nan nan
Attributes:
    Conventions:             CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf...
    calendar:                All years have exactly  365 days.
    cell_methods:            cell_methods = time: mean ==> the variable value...
    contents:                Diagnostic and Prognostic Variables
    model_doi_url:           https://doi.org/10.5065/D67H1H0V
    revision:                $Id$
    source:                  CCSM POP2, the CCSM Ocean Component
    time_period_freq:        month_1
    intake_esm_dataset_key:  ocn/ssp370/monthly/cmip6

In [7]:
tree = datatree.DataTree.from_dict(results)
tree

DataTree('None', parent=None)
├── DataTree('ocn')
│   ├── DataTree('ssp370')
│   │   └── DataTree('monthly')
│   │       ├── DataTree('cmip6')
│   │       │       Dimensions:     (member_id: 1, time: 6, z_t: 1, nlat: 384, nlon: 320, d2: 2)
│   │       │       Coordinates:
│   │       │         * member_id   (member_id) <U12 'r10i1181p1f1'
│   │       │         * time        (time) object 2015-01-16 12:00:00 ... 2015-06-16 00:00:00
│   │       │           time_bound  (time, d2) object 2015-01-01 00:00:00 ... 2015-07-01 00:00:00
│   │       │         * z_t         (z_t) float32 500.0
│   │       │       Dimensions without coordinates: nlat, nlon, d2
│   │       │       Data variables:
│   │       │           O2          (member_id, time, z_t, nlat, nlon) float32 nan nan ... nan nan
│   │       │       Attributes:
│   │       │           Conventions:             CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf...
│   │       │           calendar:                All years have exactly  365 days.
│   │       │           cell_methods:            cell_methods = time: mean ==> the variable value...
│   │       │           contents:                Diagnostic and Prognostic Variables
│   │       │           model_doi_url:           https://doi.org/10.5065/D67H1H0V
│   │       │           revision:                $Id$
│   │       │           source:                  CCSM POP2, the CCSM Ocean Component
│   │       │           time_period_freq:        month_1
│   │       │           intake_esm_dataset_key:  ocn/ssp370/monthly/cmip6
│   │       └── DataTree('smbb')
│   │               Dimensions:     (member_id: 1, time: 6, z_t: 1, nlat: 384, nlon: 320, d2: 2)
│   │               Coordinates:
│   │                 * member_id   (member_id) <U12 'r11i1231p1f2'
│   │                 * time        (time) object 2015-01-16 12:00:00 ... 2015-06-16 00:00:00
│   │                   time_bound  (time, d2) object 2015-01-01 00:00:00 ... 2015-07-01 00:00:00
│   │                 * z_t         (z_t) float32 500.0
│   │               Dimensions without coordinates: nlat, nlon, d2
│   │               Data variables:
│   │                   O2          (member_id, time, z_t, nlat, nlon) float32 nan nan ... nan nan
│   │               Attributes:
│   │                   Conventions:             CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf...
│   │                   calendar:                All years have exactly  365 days.
│   │                   cell_methods:            cell_methods = time: mean ==> the variable value...
│   │                   contents:                Diagnostic and Prognostic Variables
│   │                   model_doi_url:           https://doi.org/10.5065/D67H1H0V
│   │                   revision:                $Id$
│   │                   source:                  CCSM POP2, the CCSM Ocean Component
│   │                   time_period_freq:        month_1
│   │                   intake_esm_dataset_key:  ocn/ssp370/monthly/smbb
│   └── DataTree('historical')
│       └── DataTree('monthly')
│           ├── DataTree('smbb')
│           │       Dimensions:     (member_id: 1, time: 6, z_t: 1, nlat: 384, nlon: 320, d2: 2)
│           │       Coordinates:
│           │         * member_id   (member_id) <U12 'r11i1231p1f2'
│           │         * time        (time) object 1850-01-16 12:00:00 ... 1850-06-16 00:00:00
│           │           time_bound  (time, d2) object 1850-01-01 00:00:00 ... 1850-07-01 00:00:00
│           │         * z_t         (z_t) float32 500.0
│           │       Dimensions without coordinates: nlat, nlon, d2
│           │       Data variables:
│           │           O2          (member_id, time, z_t, nlat, nlon) float32 nan nan ... nan nan
│           │       Attributes:
│           │           Conventions:             CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf...
│           │           calendar:                All years have exactly  365 days.
│           │           cell_methods:            cell_methods

In [8]:
print(tree)

DataTree('None', parent=None)
├── DataTree('ocn')
│   ├── DataTree('ssp370')
│   │   └── DataTree('monthly')
│   │       ├── DataTree('cmip6')
│   │       │       Dimensions:     (member_id: 1, time: 6, z_t: 1, nlat: 384, nlon: 320, d2: 2)
│   │       │       Coordinates:
│   │       │         * member_id   (member_id) <U12 'r10i1181p1f1'
│   │       │         * time        (time) object 2015-01-16 12:00:00 ... 2015-06-16 00:00:00
│   │       │           time_bound  (time, d2) object 2015-01-01 00:00:00 ... 2015-07-01 00:00:00
│   │       │         * z_t         (z_t) float32 500.0
│   │       │       Dimensions without coordinates: nlat, nlon, d2
│   │       │       Data variables:
│   │       │           O2          (member_id, time, z_t, nlat, nlon) float32 nan nan ... nan nan
│   │       │       Attributes:
│   │       │           Conventions:             CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf...
│   │       │           calendar:                All years have exactly  365

In [9]:
tree.to_netcdf(
    "/home/andersy005/devel/xarray-contrib/datatree/datasets/cesm2-lens-sample.nc", mode="w"
)

In [12]:
d = datatree.open_datatree(
    "/home/andersy005/devel/xarray-contrib/datatree/datasets/cesm2-lens-sample.nc", engine="netcdf4"
)
print(d)

DataTree('None', parent=None)
├── DataTree('ocn')
│   ├── DataTree('ssp370')
│   │   └── DataTree('monthly')
│   │       ├── DataTree('cmip6')
│   │       │       Dimensions:     (member_id: 1, time: 6, z_t: 1, nlat: 384, nlon: 320, d2: 2)
│   │       │       Coordinates:
│   │       │         * member_id   (member_id) object 'r10i1181p1f1'
│   │       │         * time        (time) object 2015-01-16 12:00:00 ... 2015-06-16 00:00:00
│   │       │           time_bound  (time, d2) object ...
│   │       │         * z_t         (z_t) float32 500.0
│   │       │       Dimensions without coordinates: nlat, nlon, d2
│   │       │       Data variables:
│   │       │           O2          (member_id, time, z_t, nlat, nlon) float32 ...
│   │       │       Attributes:
│   │       │           Conventions:             CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf...
│   │       │           calendar:                All years have exactly  365 days.
│   │       │           cell_methods:          

In [10]:
%load_ext watermark
%watermark --iversions

datatree: 0.0.9
intake  : 0.6.5

